In [ ]:
import openpyxl
import xlrd
from openpyxl import Workbook
from openpyxl.styles import NamedStyle
from datetime import datetime
import pandas as pd
from docxtpl import DocxTemplate
import win32com.client
import os
from pptx import Presentation
from pptx.util import Inches


# Step 3.1: create the ppt slides

import pandas as pd
from pptx import Presentation
from pptx.util import Inches, Pt
from pptx.dml.color import RGBColor
from pptx.enum.text import PP_ALIGN
from pptx.oxml import parse_xml

# Load the Excel file
file_path = 'selected_columns_V6.xlsx' 
excel_data = pd.ExcelFile(file_path)

# Function to convert DataFrame to list of lists (including headers)
def convert_df_to_list(df):
    # Clean up text by removing line breaks, extra spaces, and "_x000D_" artifacts
    df = df.map(lambda x: x.replace('_x000D_', '').replace('\n', ' ').replace('\r', ' ').strip() if isinstance(x, str) else x)
    return [df.columns.tolist()] + df.fillna('').values.tolist()

# Load and convert each sheet's data, keeping the header row
sheets_data = {sheet: convert_df_to_list(excel_data.parse(sheet)) for sheet in excel_data.sheet_names}

# Function to estimate row height based on text content
def estimate_row_height(text, base_height=0.25, char_per_line=65):
    if isinstance(text, str):
        line_count = (len(text) // char_per_line) + 1
        return base_height * line_count
    return base_height

# Function to calculate the number of rows that can fit on a slide based on content
def calculate_rows_per_slide(data, row_height_func, max_height):
    rows = []
    current_height = 0
    for row in data[1:]:  # Skip header row
        row_height = max([row_height_func(cell) for cell in row])
        if current_height + row_height > max_height:
            break
        rows.append(row)
        current_height += row_height
    return len(rows)

# Function to split data into chunks based on the calculated rows per slide
def split_data_to_fit(data, rows_per_slide):
    return [data[:1] + data[i:i + rows_per_slide] for i in range(1, len(data), rows_per_slide)]

# Define the slide and table dimensions
slide_height = 7.0  # Total slide height in inches
title_height = 0.1  # Height reserved for the title in inches
margin_height = 0.8  # Additional margin height in inches
available_height = slide_height - title_height - margin_height

# Estimate rows per slide for each sheet
split_data = {}
for sheet, data in sheets_data.items():
    rows_per_slide = calculate_rows_per_slide(data, estimate_row_height, available_height)
    split_data[sheet] = split_data_to_fit(data, rows_per_slide)

def set_table_cell_border(cell, color):
    """Set the border of a table cell to the specified color."""
    tc = cell._tc
    tcPr = tc.get_or_add_tcPr()
    for border in ['a:lnL', 'a:lnR', 'a:lnT', 'a:lnB']:
        border_elm = parse_xml(
            f'<{border} w="12700" xmlns:a="http://schemas.openxmlformats.org/drawingml/2006/main">'
            f'<a:solidFill><a:srgbClr val="{color}"/></a:solidFill></{border}>')
        tcPr.append(border_elm)

def create_table(slide, title_text, data, column_widths):
    """Create a table on a slide with proper formatting."""
    # Set the slide title
    title = slide.shapes.title
    title.text = title_text
    title.text_frame.paragraphs[0].font.color.rgb = RGBColor(255, 98, 0)
    title.text_frame.paragraphs[0].font.size = Pt(27)
    title.text_frame.paragraphs[0].font.bold = True
    title.text_frame.paragraphs[0].alignment = PP_ALIGN.LEFT

    # Set table position and size (adjusted to reduce space between title and table)
    left = Inches(0.4)
    top = Inches(0.5)
    width = Inches(10.0)
    height = prs.slide_height - top - Inches(0.5)  # Adjust height to maintain 0.8-inch distance from the bottom

    # Add the table
    rows, cols = len(data), len(data[0])
    table = slide.shapes.add_table(rows, cols, left, top, width, Inches(0.35) * rows).table

    # Set column widths
    for i, width in enumerate(column_widths):
        table.columns[i].width = width

    # Set header row
    for i, column_name in enumerate(data[0]):
        cell = table.cell(0, i)
        cell.text = column_name
        p = cell.text_frame.paragraphs[0]
        p.font.bold = True
        p.font.size = Pt(8)
        p.font.name = 'ING Me (body)'
        p.font.color.rgb = RGBColor(0, 0, 0)  # Changed from orange to black
        p.alignment = PP_ALIGN.CENTER
        set_table_cell_border(cell, "FF6200")
        cell.fill.background()

    # Fill in the table rows
    for row_idx, row_data in enumerate(data[1:]):  # Start from the second row
        for col_idx, value in enumerate(row_data):
            cell = table.cell(row_idx + 1, col_idx)
            cell.text = str(value)
            p = cell.text_frame.paragraphs[0]
            p.font.size = Pt(7)
            p.font.name = 'ING Me (body)'
            p.font.color.rgb = RGBColor(0, 0, 0)
            p.alignment = PP_ALIGN.LEFT if col_idx > 0 else PP_ALIGN.CENTER
            set_table_cell_border(cell, "FF6200")
            cell.fill.background()

def add_slides_with_table(prs, title, data, column_widths):
    """Add slides with a table split over multiple slides if necessary."""
    for chunk in data:
        slide_layout = prs.slide_layouts[5]
        slide = prs.slides.add_slide(slide_layout)
        create_table(slide, title, chunk, column_widths)

def is_empty_slide(slide):
    """Check if a slide is empty (no significant shapes)."""
    for shape in slide.shapes:
        if shape.has_text_frame and shape.text.strip():
            return False
        if shape.has_table:
            return False
    return True

def delete_empty_slides(prs):
    """Remove empty slides from the presentation."""
    slide_indexes_to_delete = []
    for i, slide in enumerate(prs.slides):
        if is_empty_slide(slide):
            slide_indexes_to_delete.append(i)

    for i in sorted(slide_indexes_to_delete, reverse=True):
        rId = prs.slides._sldIdLst[i].rId
        prs.part.drop_rel(rId)
        del prs.slides._sldIdLst[i]

# Load the presentation
prs = Presentation()

# Define the column widths (adjust these based on the data)
high_columns_widths = [
    Inches(1.5), Inches(2.5), Inches(0.7), Inches(0.6), Inches(0.9), Inches(0.9), Inches(1.1), Inches(0.8)
]

medium_low_columns_widths = [
    Inches(1.5), Inches(2.0), Inches(0.7), Inches(0.8), Inches(0.6), Inches(0.9), Inches(0.9), Inches(1.1), Inches(0.8)
]

accepted_columns_widths = [
    Inches(1.8), Inches(2.0), Inches(0.7), Inches(1.2), Inches(1.0), Inches(1.0), Inches(1.0)
]

# Add slides with tables for each category
add_slides_with_table(prs, "High and critical Findings", split_data['High'], high_columns_widths)
add_slides_with_table(prs, "Medium and low Findings", split_data['Medium_Low'], medium_low_columns_widths)
add_slides_with_table(prs, "Accepted Risks Cloud & DCN", split_data['Accepted'], accepted_columns_widths)

# Remove any empty slides
delete_empty_slides(prs)

# Save the presentation
prs.save('Cloud Compliance steerco.pptx')

# Step 3.2: insert the docs files

# Function to add a DOCX attachment in the "Issue Description" column
import win32com.client
import os

# Step 5: Insert DOCX files into the PowerPoint presentation
def insert_docx_as_icon_in_ppt(ppt_path, high_docx_files, medium_low_docx_files, issue_description_col_idx=7):
    App = win32com.client.Dispatch("PowerPoint.Application")
    App.Visible = True

    Presentation = App.Presentations.Open(os.path.abspath(ppt_path))

    docx_index_high = 0
    docx_index_medium_low = 0

    for slide in Presentation.Slides:
        title_shape = slide.Shapes.Title
        if title_shape:
            title_text = title_shape.TextFrame.TextRange.Text
        else:
            title_text = ""

        for shape in slide.Shapes:
            if shape.HasTable:
                table = shape.Table

                for row_idx in range(2, table.Rows.Count + 1):
                    if "Medium and low Findings" in title_text and docx_index_medium_low < len(medium_low_docx_files):
                        docx_file_path = medium_low_docx_files[docx_index_medium_low]
                        docx_index_medium_low += 1
                        cell = table.Cell(row_idx, issue_description_col_idx)
                        left = cell.Shape.left + 10  # Adjusted position to move closer to the column
                        top = cell.Shape.Top + 10
                        width = 350  # Custom width for Medium and Low Findings
                        height = 45
                    elif "High and critical Findings" in title_text and docx_index_high < len(high_docx_files):
                        docx_file_path = high_docx_files[docx_index_high]
                        docx_index_high += 1
                        cell = table.Cell(row_idx, issue_description_col_idx)
                        left = cell.Shape.left + 20  # Original position for High and Critical Findings
                        top = cell.Shape.Top + 10
                        width = 210  # Default width for High and Critical Findings
                        height = 45
                    else:
                        continue

                    # Insert DOCX as an OLE object (icon)
                    ole_shape = slide.Shapes.AddOLEObject(
                        Left=left,
                        Top=top,
                        Width=width,
                        Height=height,
                        FileName=os.path.abspath(docx_file_path),
                        DisplayAsIcon=True,
                        IconFileName="C:\\Program Files\\Microsoft Office\\root\\Office16\\WINWORD.EXE",
                        IconLabel="Microsoft Word Document"
                    )

                    # Adjust the icon size using ScaleWidth and ScaleHeight instead of setting exact dimensions
                    ole_shape.LockAspectRatio = False
                    ole_shape.ScaleWidth(0.5, True)  # Scale the width by 0.5x
                    ole_shape.ScaleHeight(0.5, True)  # Scale the height by 0.5x

    Presentation.Save()
    # Presentation.Close()
    App.Quit()

# Define paths
ppt_path = 'Cloud Compliance steerco.pptx'
high_docx_files = [f"High_generated_doc_{i}.docx" for i in range(1, 12)]  # Adjust paths accordingly
medium_low_docx_files = [f"Medium_Low_generated_doc_{i}.docx" for i in range(1, 20)]  # Adjust paths accordingly

insert_docx_as_icon_in_ppt(ppt_path, high_docx_files, medium_low_docx_files)

# Final Step: Adjust the pptx file to fit the screen

from pptx import Presentation
from pptx.util import Inches
from pptx.enum.text import PP_ALIGN

# Load the PowerPoint presentation
presentation = Presentation('Cloud Compliance steerco.pptx')


# Iterate through each slide
for slide in presentation.slides:
    # Find the title placeholder
    for shape in slide.shapes:
        if shape.is_placeholder and shape.placeholder_format.idx == 0:  # Title placeholder
            # Set the title placeholder's position to the top of the slide
            shape.top = Inches(0.3)  # Adjust as needed to move the title to the top
            shape.left = Inches(0.5)  # Adjust as needed to center the title horizontally
            shape.width = presentation.slide_width - Inches(1)  # Adjust width to fit the slide

            # Set the title text alignment to center
            shape.text_frame.paragraphs[0].alignment = PP_ALIGN.LEFT


# Save the modified presentation
presentation.save('Cloud Compliance steerco.pptx')
